# Retrieval Augmented Generation (RAG)

Retrieval Augmented Generation (RAG) is a natural language processing (NLP) technique that combines the strengths of retrieval-based and generation-based models. It enhances traditional language generation models by incorporating a retrieval mechanism to fetch relevant context from a large corpus of text before generating responses. This approach allows RAG models to produce more accurate and contextually relevant outputs.

#### Mathematical Formulation:

The RAG model generates answers using the following formula:

Answer = $\arg\max_{a} P(a|q, C)$]

Where:
- Answer represents the answer generated by the model.
- $\arg\max_{a}$ denotes the argument that maximizes the function with respect to \( a \).
- $P(a|q, C)$ represents the probability of generating answer \( a \) given the query \( q \) and retrieved context \( C \).

#### Business Scenario Example:

In a cybersecurity context, suppose a security analyst wants to gather information about Zscaler, a cloud-based cybersecurity company, to understand its offerings and capabilities. Using RAG, the analyst can input a query such as "What are the key features of Zscaler's cloud security platform?" The RAG model then retrieves relevant information from large text datasets, such as the Stanford Question Answering Dataset, Wikipedia, or industry reports. Finally, based on the retrieved context, the RAG model generates a comprehensive answer that highlights the key features of Zscaler's platform.

**Note:** This example illustrates a simplified scenario. In practice, RAG can leverage much larger text datasets, such as the Stanford Question Answering Dataset, Wikipedia, or domain-specific corpora, to generate accurate and informative responses.

In [4]:
from transformers import DPRQuestionEncoderTokenizer, DPRContextEncoder, DPRQuestionEncoder, DPRContextEncoderTokenizer
import torch
import transformers
import logging
import warnings
# Suppress warnings from the Transformers library
transformers.logging.set_verbosity_error()
# Suppress all warnings (including FutureWarning, etc.)
warnings.filterwarnings('ignore')
# Optionally, set the Python logging module to ERROR level
logging.getLogger().setLevel(logging.ERROR)

# Load the tokenizers and models
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

# Define the updated question
question = "What were the key factors in Zscaler's growth and market success after its founding?"

# Define the updated passages
passages = [
    "Zscaler is a cloud-based cybersecurity company that offers a comprehensive platform for secure internet access and digital transformation. It provides cloud security solutions designed to protect users, data, and applications from various cyber threats, regardless of their location or device. Zscaler's platform operates by routing traffic through its global network of data centers, where it applies security policies and performs real-time threat analysis to detect and prevent cyber attacks. Key features of Zscaler include web security, cloud application security, zero trust network access, data protection, and threat intelligence. By offering a unified and scalable security solution, Zscaler helps organizations enhance their security posture while enabling secure and efficient digital experiences for users.",
    "Founded in 2007 by Jay Chaudhry and K. Kailash, Zscaler's mission was to transform network security for the modern cloud era. The company went public in March 2018, highlighting its rapid growth and the increasing demand for cloud-native security solutions.",
    "Zscaler has shown remarkable growth since its IPO, with its cloud security solutions being adopted by thousands of companies worldwide, including several Fortune 500 firms. The company's focus on innovation and expanding its security platform has driven its market valuation significantly.",
    "Zscaler invests heavily in research and development to stay ahead of emerging cybersecurity threats. Its proprietary technologies, such as the Zscaler Zero Trust Exchange, leverage artificial intelligence and machine learning to provide enhanced security measures."
]

# Tokenize and encode the question
question_inputs = question_tokenizer(question, return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    question_embedding = question_encoder(**question_inputs).pooler_output

# Tokenize and encode each passage
passage_embeddings = []
for passage in passages:
    passage_inputs = context_tokenizer(passage, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        passage_embedding = context_encoder(**passage_inputs).pooler_output
        passage_embeddings.append(passage_embedding)

# Stack embeddings for similarity calculation
passage_embeddings = torch.stack(passage_embeddings).squeeze(1)

# Calculate cosine similarity scores between question and each passage embedding
similarity_scores = torch.nn.functional.cosine_similarity(question_embedding, passage_embeddings, dim=-1)

# Identify the most similar passage
max_similarity_score, max_similarity_idx = torch.max(similarity_scores, dim=0)

# Output the most relevant passage and its similarity score
print("Question:", question)
print("Most similar passage:", passages[max_similarity_idx.item()])
print("Similarity score:", max_similarity_score.item())

Question: What were the key factors in Zscaler's growth and market success after its founding?
Most similar passage: Zscaler has shown remarkable growth since its IPO, with its cloud security solutions being adopted by thousands of companies worldwide, including several Fortune 500 firms. The company's focus on innovation and expanding its security platform has driven its market valuation significantly.
Similarity score: 0.6895880103111267
